In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import numpy as np
import math


In [3]:
# This is the derived equation for acceleration
def get_a_vacm(v,params):
    m,c,h,v_o,h = params   #Not used, just for consistancy
    
    a = 9.81    #gravitational acceleration(constant near earth)
    
    return a

In [4]:
# This is the derived equation for acceleration through a viscous liquid

def get_a_visc(v, params):
    m,c,h,v_o,h = params
    
    g = 9.81   #gravitational Constant
    
    drag = c * v / m
    
    if drag > g:
        print("Houston, we have a problem!")
        
    else:
        # looking good, keeping moving along
        pass
    
    a = g - drag
    
    return a

In [5]:
# This is an implementation of euler method for integrating an initial value problem
def euler(ode, params):
    m,c,h,v_o,h = params
    
    t = np.arange(0,10,h)
    v = np.zeros(len(t))
    v_int = np.zeros(len(t))
    v[0] = v_o
    
    for i in range(len(t)):
        if i == 0:
            v_int[i] = v_o
        else:
            a = ode(v_int[i-1],params)
            v_int[i] = v_int[i-1] + a * h
            v[i] = v_int[i]
            
    return v

In [64]:
def get_real_sol(ode,params):
    m,c,h,v_o,h = params

    t = np.arange(0,10,h)
    v = np.zeros(len(t))
    
    if ode == 'viscous':
        vFun = lambda v: v + 9.81 * m / c * (1.0 - np.exp(-c * h / m))

    if ode == 'vacuum':
        vFun = lambda v: v+ 9.81 * h
   
    v[0] = v_o
    
    for i in range(1,len(t)):
        v[i] = vFun(v[i-1])
    
    return v

In [65]:
# This returns which ode Euler's method will use
def get_ode(ode):
    if ode is 'vacuum':
        ode = get_a_vacm
    else:
        ode = get_a_visc 
      
    return ode

In [66]:
def KeyFun(v):
    for i in range(1,len(v)):
        if v[i-1] > 0 and v[i] < 0:
            print('Key = EIFB')
            break
        

In [67]:
# This function simply calls those above and plots the computed data
def plot_v(m, c, h, v_o, ODE):
    
    ode = get_ode(ODE)
    params = [m,c,h,v_o,h]

    v_real = get_real_sol(ODE,params)
    v_approx = euler(ode,params)
    error = abs(v_real - v_approx)
    
    KeyFun(v_approx)
    
    t_discrt = np.arange(0,10+h,h)
    
    # plot both datasets
    plot_v = go.Scatter(name='solution', x=t_discrt, y=v_real)
    plot_v_approx = go.Scatter(name='euler approx', x=t_discrt, y=v_approx)
        
    iplot([plot_v, plot_v_approx])
    
    

In [68]:
# Define the variables that can be interacted with
w = interact(plot_v, m=(1, 10, 1), c=(0.0, 1, 0.1), h=(0.05, 2, 0.05), v_o = (-5,5,1), ODE={'vacuum','viscous'})

interactive(children=(IntSlider(value=5, description='m', max=10, min=1), FloatSlider(value=0.5, description='…